%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Computational drug discovery: part 2

Exploratory Data Analysis

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [53]:
# Install conda and rdkit
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-08-28 11:37:33--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.2’

Miniconda3-py37_4.8 100%[===================>]  81.12M   181MB/s    in 0.4s    

2021-08-28 11:37:33 (181 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.2’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: \ | / - \ | / - \ | / - \ | / failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: - \ | / - \ 

In [54]:
# Import libraries
import pandas as pd

In [93]:
! wget -O genital_herpes_03_bioactivity_data_curated.csv https://raw.githubusercontent.com/AnVales/Drug-discovery/main/genital_herpes_03_bioactivity_data_curated.csv

--2021-08-28 12:12:26--  https://raw.githubusercontent.com/AnVales/Drug-discovery/main/genital_herpes_03_bioactivity_data_curated.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21267 (21K) [text/plain]
Saving to: ‘genital_herpes_03_bioactivity_data_curated.csv’

genital_herpes_03_b 100%[===================>]  20.77K  --.-KB/s    in 0s      

2021-08-28 12:12:26 (76.2 MB/s) - ‘genital_herpes_03_bioactivity_data_curated.csv’ saved [21267/21267]



In [94]:
bioactivity_data = pd.read_csv('genital_herpes_03_bioactivity_data_curated.csv')
bioactivity_data

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity
0,CHEMBL122071,CC(=O)O[C@@H]1CC(=O)N1C(=O)NC(C)C,100000.0,inactive
1,CHEMBL124107,CCCCNC(=O)N1C(=O)C[C@H]1OC(C)=O,68000.0,inactive
2,CHEMBL120853,C[C@@H]1C(=O)N(C(=O)NCc2ccccc2)[C@@H]1Oc1ccc(C...,100000.0,inactive
3,CHEMBL122296,CC(=O)O[C@@H]1[C@@H](C)C(=O)N1C(=O)NCc1ccccc1,6500.0,intermediate
4,CHEMBL333734,CC(=O)O[C@@H]1[C@H](C)C(=O)N1C(=O)NCc1ccccc1,27000.0,inactive
...,...,...,...,...
230,CHEMBL107357,Cc1csc2nc(Cc3ccc(C(=O)c4ccc(CO)cc4)cc3)oc(=O)c12,940.0,intermediate
231,CHEMBL322676,Cc1csc2nc(Cc3cccc(NC(=O)c4cccc(NO)c4)c3)oc(=O)c12,45.0,intermediate
232,CHEMBL104662,Cc1csc2nc(Cc3ccc(C(=O)c4ccccc4)cc3)oc(=O)c12,800.0,inactive
233,CHEMBL106921,Cc1csc2nc(Cc3cccc(NC(=O)c4ccccc4)c3)oc(=O)c12,1500.0,active


In [95]:
bioactivity_data.shape

(235, 4)

In [96]:
bioactivity_data_clean = bioactivity_data.dropna(how='any')
bioactivity_data_clean

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity
0,CHEMBL122071,CC(=O)O[C@@H]1CC(=O)N1C(=O)NC(C)C,100000.0,inactive
1,CHEMBL124107,CCCCNC(=O)N1C(=O)C[C@H]1OC(C)=O,68000.0,inactive
2,CHEMBL120853,C[C@@H]1C(=O)N(C(=O)NCc2ccccc2)[C@@H]1Oc1ccc(C...,100000.0,inactive
3,CHEMBL122296,CC(=O)O[C@@H]1[C@@H](C)C(=O)N1C(=O)NCc1ccccc1,6500.0,intermediate
4,CHEMBL333734,CC(=O)O[C@@H]1[C@H](C)C(=O)N1C(=O)NCc1ccccc1,27000.0,inactive
...,...,...,...,...
230,CHEMBL107357,Cc1csc2nc(Cc3ccc(C(=O)c4ccc(CO)cc4)cc3)oc(=O)c12,940.0,intermediate
231,CHEMBL322676,Cc1csc2nc(Cc3cccc(NC(=O)c4cccc(NO)c4)c3)oc(=O)c12,45.0,intermediate
232,CHEMBL104662,Cc1csc2nc(Cc3ccc(C(=O)c4ccccc4)cc3)oc(=O)c12,800.0,inactive
233,CHEMBL106921,Cc1csc2nc(Cc3cccc(NC(=O)c4ccccc4)c3)oc(=O)c12,1500.0,active


In [104]:
bioactivity_data_clean_index = bioactivity_data_clean.reset_index(drop=True)
bioactivity_data_clean_index

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity
0,CHEMBL122071,CC(=O)O[C@@H]1CC(=O)N1C(=O)NC(C)C,100000.0,inactive
1,CHEMBL124107,CCCCNC(=O)N1C(=O)C[C@H]1OC(C)=O,68000.0,inactive
2,CHEMBL120853,C[C@@H]1C(=O)N(C(=O)NCc2ccccc2)[C@@H]1Oc1ccc(C...,100000.0,inactive
3,CHEMBL122296,CC(=O)O[C@@H]1[C@@H](C)C(=O)N1C(=O)NCc1ccccc1,6500.0,intermediate
4,CHEMBL333734,CC(=O)O[C@@H]1[C@H](C)C(=O)N1C(=O)NCc1ccccc1,27000.0,inactive
...,...,...,...,...
227,CHEMBL107357,Cc1csc2nc(Cc3ccc(C(=O)c4ccc(CO)cc4)cc3)oc(=O)c12,940.0,intermediate
228,CHEMBL322676,Cc1csc2nc(Cc3cccc(NC(=O)c4cccc(NO)c4)c3)oc(=O)c12,45.0,intermediate
229,CHEMBL104662,Cc1csc2nc(Cc3ccc(C(=O)c4ccccc4)cc3)oc(=O)c12,800.0,inactive
230,CHEMBL106921,Cc1csc2nc(Cc3cccc(NC(=O)c4ccccc4)c3)oc(=O)c12,1500.0,active


In [105]:
bioactivity_data_clean_index[bioactivity_data_clean_index.isnull().any(1)]

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity


In [29]:
# Rule-of-Five or Lipinski's Rule.
# Evaluation of the druglikeness of compounds.
# pharmacokinetic profile: Absorption, Distribution, Metabolism and Excretion (ADME)

# Molecular weight < 500 Dalton
# Octanol-water partition coefficient (LogP) < 5
# Hydrogen bond donors < 5
# Hydrogen bond acceptors < 10

In [99]:
# Import libraries
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [100]:
# Calculate descriptors
# Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation

def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [106]:
bioactivity_data_smiles_lipinski = lipinski(bioactivity_data_clean_index.canonical_smiles)
bioactivity_data_smiles_lipinski

,MW,LogP,NumHDonors,NumHAcceptors
0,214.221,0.22590,1.0,4.0
1,228.248,0.61760,1.0,4.0
2,354.362,2.47780,2.0,4.0
3,276.292,1.26370,1.0,4.0
4,276.292,1.26370,1.0,4.0
...,...,...,...,...
227,391.448,3.87202,1.0,6.0
228,407.451,4.20212,3.0,7.0
229,361.422,4.37972,0.0,5.0
230,376.437,4.40102,1.0,5.0


In [ ]:
# We have Lipinski's properties now but we don't know to which it remains.  We need to link the data.
# bioactivity_data_smiles: molecule_chembl_id, standard_value, bioactivity, canonical_smiles
# bioactivity_data_smiles_lipinski: MW, LogP, NumHDonors, NumHAcceptors

In [109]:
bioactivity_data_combined = pd.concat([bioactivity_data_clean_index,bioactivity_data_smiles_lipinski], axis=1)
bioactivity_data_combined

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity,MW,LogP,NumHDonors,NumHAcceptors
0,CHEMBL122071,CC(=O)O[C@@H]1CC(=O)N1C(=O)NC(C)C,100000.0,inactive,214.221,0.22590,1.0,4.0
1,CHEMBL124107,CCCCNC(=O)N1C(=O)C[C@H]1OC(C)=O,68000.0,inactive,228.248,0.61760,1.0,4.0
2,CHEMBL120853,C[C@@H]1C(=O)N(C(=O)NCc2ccccc2)[C@@H]1Oc1ccc(C...,100000.0,inactive,354.362,2.47780,2.0,4.0
3,CHEMBL122296,CC(=O)O[C@@H]1[C@@H](C)C(=O)N1C(=O)NCc1ccccc1,6500.0,intermediate,276.292,1.26370,1.0,4.0
4,CHEMBL333734,CC(=O)O[C@@H]1[C@H](C)C(=O)N1C(=O)NCc1ccccc1,27000.0,inactive,276.292,1.26370,1.0,4.0
...,...,...,...,...,...,...,...,...
227,CHEMBL107357,Cc1csc2nc(Cc3ccc(C(=O)c4ccc(CO)cc4)cc3)oc(=O)c12,940.0,intermediate,391.448,3.87202,1.0,6.0
228,CHEMBL322676,Cc1csc2nc(Cc3cccc(NC(=O)c4cccc(NO)c4)c3)oc(=O)c12,45.0,intermediate,407.451,4.20212,3.0,7.0
229,CHEMBL104662,Cc1csc2nc(Cc3ccc(C(=O)c4ccccc4)cc3)oc(=O)c12,800.0,inactive,361.422,4.37972,0.0,5.0
230,CHEMBL106921,Cc1csc2nc(Cc3cccc(NC(=O)c4ccccc4)c3)oc(=O)c12,1500.0,active,376.437,4.40102,1.0,5.0


In [ ]:
# Convert IC50 to pIC50: 
# to allow IC50 data to be more uniformly distributed, we will convert IC50 to the negative logarithmic scale which is essentially -log10(IC50)


# This custom function pIC50() will accept a DataFrame as input and will:

# Take the IC50 values from the standard_value column and converts it from nM to M by multiplying the value by 10^{-9}
# Take the molar value and apply -log10
# Delete the standard_value column and create a new pIC50 column


In [110]:
# https://github.com/chaninlab/estrogen-receptor-alpha-qsar/blob/master/02_ER_alpha_RO5.ipynb
import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

In [111]:
# Values greater than 100,000,000 will be fixed at 100,000,000 otherwise the negative logarithmic value will become negative.
bioactivity_data_combined.standard_value.describe()

count        232.000000
mean       56746.834052
std       141451.585874
min            2.700000
25%          180.000000
50%         1400.000000
75%        33250.000000
max      1000000.000000
Name: standard_value, dtype: float64

In [112]:
-np.log10( (10**-9)* 100000000 )

1.0

In [113]:
-np.log10( (10**-9)* 10000000000 )

-1.0

In [114]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

In [115]:
# the values in the standard_value column is normalized
bioactivity_data_combined_norm = norm_value(bioactivity_data_combined)
bioactivity_data_combined_norm

,molecule_chembl_id,canonical_smiles,bioactivity,MW,LogP,NumHDonors,NumHAcceptors,standard_value_norm
0,CHEMBL122071,CC(=O)O[C@@H]1CC(=O)N1C(=O)NC(C)C,inactive,214.221,0.22590,1.0,4.0,100000.0
1,CHEMBL124107,CCCCNC(=O)N1C(=O)C[C@H]1OC(C)=O,inactive,228.248,0.61760,1.0,4.0,68000.0
2,CHEMBL120853,C[C@@H]1C(=O)N(C(=O)NCc2ccccc2)[C@@H]1Oc1ccc(C...,inactive,354.362,2.47780,2.0,4.0,100000.0
3,CHEMBL122296,CC(=O)O[C@@H]1[C@@H](C)C(=O)N1C(=O)NCc1ccccc1,intermediate,276.292,1.26370,1.0,4.0,6500.0
4,CHEMBL333734,CC(=O)O[C@@H]1[C@H](C)C(=O)N1C(=O)NCc1ccccc1,inactive,276.292,1.26370,1.0,4.0,27000.0
...,...,...,...,...,...,...,...,...
227,CHEMBL107357,Cc1csc2nc(Cc3ccc(C(=O)c4ccc(CO)cc4)cc3)oc(=O)c12,intermediate,391.448,3.87202,1.0,6.0,940.0
228,CHEMBL322676,Cc1csc2nc(Cc3cccc(NC(=O)c4cccc(NO)c4)c3)oc(=O)c12,intermediate,407.451,4.20212,3.0,7.0,45.0
229,CHEMBL104662,Cc1csc2nc(Cc3ccc(C(=O)c4ccccc4)cc3)oc(=O)c12,inactive,361.422,4.37972,0.0,5.0,800.0
230,CHEMBL106921,Cc1csc2nc(Cc3cccc(NC(=O)c4ccccc4)c3)oc(=O)c12,active,376.437,4.40102,1.0,5.0,1500.0


In [116]:
bioactivity_data_combined_norm.standard_value_norm.describe()

count        232.000000
mean       56746.834052
std       141451.585874
min            2.700000
25%          180.000000
50%         1400.000000
75%        33250.000000
max      1000000.000000
Name: standard_value_norm, dtype: float64

In [117]:
bioactivity_data_final = pIC50(bioactivity_data_combined_norm)
bioactivity_data_final

,molecule_chembl_id,canonical_smiles,bioactivity,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL122071,CC(=O)O[C@@H]1CC(=O)N1C(=O)NC(C)C,inactive,214.221,0.22590,1.0,4.0,4.000000
1,CHEMBL124107,CCCCNC(=O)N1C(=O)C[C@H]1OC(C)=O,inactive,228.248,0.61760,1.0,4.0,4.167491
2,CHEMBL120853,C[C@@H]1C(=O)N(C(=O)NCc2ccccc2)[C@@H]1Oc1ccc(C...,inactive,354.362,2.47780,2.0,4.0,4.000000
3,CHEMBL122296,CC(=O)O[C@@H]1[C@@H](C)C(=O)N1C(=O)NCc1ccccc1,intermediate,276.292,1.26370,1.0,4.0,5.187087
4,CHEMBL333734,CC(=O)O[C@@H]1[C@H](C)C(=O)N1C(=O)NCc1ccccc1,inactive,276.292,1.26370,1.0,4.0,4.568636
...,...,...,...,...,...,...,...,...
227,CHEMBL107357,Cc1csc2nc(Cc3ccc(C(=O)c4ccc(CO)cc4)cc3)oc(=O)c12,intermediate,391.448,3.87202,1.0,6.0,6.026872
228,CHEMBL322676,Cc1csc2nc(Cc3cccc(NC(=O)c4cccc(NO)c4)c3)oc(=O)c12,intermediate,407.451,4.20212,3.0,7.0,7.346787
229,CHEMBL104662,Cc1csc2nc(Cc3ccc(C(=O)c4ccccc4)cc3)oc(=O)c12,inactive,361.422,4.37972,0.0,5.0,6.096910
230,CHEMBL106921,Cc1csc2nc(Cc3cccc(NC(=O)c4ccccc4)c3)oc(=O)c12,active,376.437,4.40102,1.0,5.0,5.823909


In [118]:
bioactivity_data_final.pIC50.describe()

count    232.000000
mean       5.796134
std        1.481253
min        3.000000
25%        4.478245
50%        5.853872
75%        6.744727
max        8.568636
Name: pIC50, dtype: float64

In [119]:
# Save all these data in a csv file
bioactivity_data_final.to_csv('genital_herpes_04_bioactivity_data_3class_pIC50.csv')

In [122]:
# Remove the 'intermediate' bioactivity class
bioactivity_data_2class = bioactivity_data_final[bioactivity_data_final['class'] != 'intermediate']
bioactivity_data_2class

KeyError: ignored